In [ ]:
import pandas as pd
from dateutil import parser
import statsmodels.api as sm

class DID_Class:
    def __init__(self):
        self.control = None
        self.treatment = None
        self.time = None
        self.timeframe = None

    def user_input(self):
        self.control = input("Enter the control group dataframe: ")
        self.treatment = input("Enter the treatment group dataframe: ")
        self.time = str(input("Intervention time: "))
        self.timeframe = input("Enter timeframe (M for month, W for week, Y for Year, or leave empty for day): ").upper()
        if self.timeframe not in ['M', 'W', 'Y']:
            self.timeframe = 'D'

    def time_check(self):
        count_treat_time = 0
        count_treat_int = 0
        count_control_time = 0
        count_control_int = 0
        for columns in self.treatment.columns:
            if pd.api.types.is_datetime64_any_dtype(self.treatment[columns]):
                treatment_timeframe = self.treatment[columns]
                count_treat_time += 1
                if count_treat_time > 1:
                    raise ValueError("treatment data frame contains more than one datetime column")
            elif pd.api.types.is_integer64_dtype(self.treatment[columns]):
                treatment_target = self.treatment[columns]
                count_treat_int += 1
                if count_treat_int > 1:
                    raise ValueError("treatment data frame contains more than one integer column")

        for columns in self.control.columns:
            if pd.api.types.is_datetime64_any_dtype(self.control[columns]):
                control_timeframe = self.control[columns]
                count_control_time += 1
                if count_control_time > 1:
                    raise ValueError("control data frame contains more than one datetime column")
            elif pd.api.types.is_integer64_dtype(self.control[columns]):
                control_target = self.control[columns]
                count_control_int += 1
                if count_control_int > 1:
                    raise ValueError("control data frame contains more than one integer column")

        treatment_dataframe = pd.DataFrame({
            'timeframe': treatment_timeframe,
            'target': treatment_target
        })

        control_dataframe = pd.DataFrame({
            'timeframe': control_timeframe,
            'target': control_target
        })

        return treatment_dataframe, control_dataframe

    def dataframe_cleaning(self, treatment_dataframe, control_dataframe):
        treatment_dataframe['timeframe'] = pd.to_datetime(treatment_dataframe['timeframe'])
        treatment_dataframe['Week'] = treatment_dataframe['timeframe'].dt.week
        treatment_dataframe['Month'] = treatment_dataframe['timeframe'].dt.month
        treatment_dataframe['Year'] = treatment_dataframe['timeframe'].dt.year

        control_dataframe['timeframe'] = pd.to_datetime(control_dataframe['timeframe'])
        control_dataframe['Week'] = control_dataframe['timeframe'].dt.week
        control_dataframe['Month'] = control_dataframe['timeframe'].dt.month
        control_dataframe['Year'] = control_dataframe['timeframe'].dt.year

    def group_by(self):
        if self.timeframe == 'Y':
            control_dataframe_groupby = control_dataframe.groupby('Year')['control_target'].mean()
            treatment_dataframe_groupby = treatment_dataframe.groupby('Year')['treatment_target'].mean()
        elif self.timeframe == 'M':
            control_dataframe_groupby = control_dataframe.groupby('Month')['control_target'].mean()
            treatment_dataframe_groupby = treatment_dataframe.groupby('Month')['treatment_target'].mean()
        elif self.timeframe == 'W':
            control_dataframe_groupby = control_dataframe.groupby('Week')['control_target'].mean()
            treatment_dataframe_groupby = treatment_dataframe.groupby('Week')['treatment_target'].mean()
        else:
            control_dataframe_groupby = control_dataframe
            treatment_dataframe_groupby = treatment_dataframe
        return control_dataframe_groupby, treatment_dataframe_groupby

    def start_date(self):
        try:
            treat_start_date = parser.parse(self.time)
            print("Parsed date:", treat_start_date)
        except ValueError:
            raise ValueError("Invalid date format")

    def creation_indicator_variables(self, control_dataframe_groupby, treatment_dataframe_groupby):
        control_timeframe = 'control_timeframe'  # Replace with the correct column name
        treatment_timeframe = 'treatment_timeframe'  # Replace with the correct column name
        treat_start_date = parser.parse(self.time)

        control_dataframe_groupby['treat'] = control_dataframe_groupby[control_timeframe].apply(
            lambda x: 0 if x < treat_start_date else 1
        )
        control_dataframe_groupby['group'] = 0
        control_dataframe_groupby['interaction'] = control_dataframe_groupby['group'] * control_dataframe_groupby[
            'treat']

        treatment_dataframe_groupby['treat'] = treatment_dataframe_groupby[treatment_timeframe].apply(
            lambda x: 0 if x < treat_start_date else 1
        )
        treatment_dataframe_groupby['group'] = 1
        treatment_dataframe_groupby['interaction'] = treatment_dataframe_groupby['group'] * treatment_dataframe_groupby[
            'treat']

    def concat_data(self, control_dataframe_groupby, treatment_dataframe_groupby):
        concat_data = pd.concat([treatment_dataframe_groupby, control_dataframe_groupby], axis=0, ignore_index=True)
        concat_data.columns = ['target', 'group', 'treat', 'interaction']
        return concat_data

    def ols_regression(self, concat_data):
        model = sm.OLS.from_formula('target ~ treat + group + interaction', data=concat_data)
        ols_result = model.fit()
        return ols_result.summary()

    def p_value(self, ols_summary):
        if ols_summary.tables[1].data[3][3] < self.los:
            print("Interaction is significant.")
        else:
            print("Interaction is not significant.")
